In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optimizers
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from tqdm import tqdm

from models import SimpleMLP
from data_loaders import KitchenDataset

In [ ]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
data_path = '../data/DEF-01.json'

# setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# HYPERPARAMETERS
HIDDEN_LAYERS = 1
HIDDEN_FEATURES = 64

BATCH_SIZE = 512
NUM_EPOCHS = 300
LR = 0.0008

In [ ]:
# get full dataset
kitchen_dataset = KitchenDataset(data_path)
#train_data = KitchenDataset(data_path, train=True, num_examples=1024)
#test_data = KitchenDataset(data_path, train=False, num_examples=4000)
#num_kitchens = train_data.num_kitchens
#num_items = train_data.num_items

# split data into training and test data
num_kitchens = kitchen_dataset.num_kitchens
num_items = kitchen_dataset.num_items
num_training_examples = int(0.8 * len(kitchen_dataset))
num_test_examples = len(kitchen_dataset) - num_training_examples

train_data, test_data = torch.utils.data.random_split(kitchen_dataset, [num_training_examples, num_test_examples])

# setup data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# initialize model and optimizer
model = SimpleMLP(num_kitchens=num_kitchens,
                  num_items=num_items,
                  hidden_layers=HIDDEN_LAYERS, 
                  hidden_features=HIDDEN_FEATURES,
                  data_means=kitchen_dataset.get_food_means()).to(device)
optim = optimizers.Adam(model.parameters(), lr=LR)
criterion = F.cross_entropy

In [ ]:
# TRAIN CLASSIFIER
print('=' * 32)
print(f'Start training')
print(f'Device: {device}')
print(f'Number of epochs: {NUM_EPOCHS}')
print('=' * 32)

losses = []
test_losses = []
accuracies = []
test_x = []

# -----------------------------------------------------------------
# MAIN TRAINING LOOP
# -----------------------------------------------------------------
prog_bar = tqdm(range(NUM_EPOCHS))
num_training_iterations = 0
for epoch in range(NUM_EPOCHS):
    # train one epoch
    for _, (data, targets) in enumerate(iter(train_loader)):
        data, targets = data.to(device), targets.to(device)
        
        prediction = model(data)
        loss = criterion(prediction, targets)
        losses.append(loss.item())
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        num_training_iterations += 1
    
    # validation
    test_loss = 0
    test_predictions = []
    true_labels = []
    with torch.no_grad():
        # iterate through test set
        for _, (data, targets) in enumerate(iter(test_loader)):
            data, targets = data.to(device), targets.to(device)
            prediction = model(data)
            test_loss += criterion(prediction, targets, reduction='sum')
            test_predictions.append(prediction)
            true_labels.append(targets)
        # average test loss
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        test_x.append(num_training_iterations)
        
        # compute accuracy
        true_test_labels = torch.argmax(torch.cat(true_labels).detach().cpu(), dim=1)
        all_predictions = torch.argmax(torch.cat(test_predictions).detach().cpu(), dim=1)
        accuracy = torch.sum(torch.where(all_predictions - true_test_labels == 0, 1, 0)) / len(test_data) * 100
        accuracies.append(accuracy)
        
    prog_bar.update(1)

prog_bar.close()    

# plot summary
fig, ax = plt.subplots(1, 2, figsize=(10,4))

fig.suptitle('Training Summary')
ax[0].plot(np.log(losses), label='Training Loss')
ax[0].plot(test_x, np.log(test_losses), label='Test Loss')
ax[0].legend()
ax[0].set_xlabel('Iterations')
ax[0].set_ylabel('log loss')
ax[0].set_title(f'Log loss over {NUM_EPOCHS} epochs')
ax[1].plot(test_x, accuracies, label='Test Accuracy')
ax[1].set_xlabel('Iterations')
ax[1].set_ylabel('Accuracy %')
ax[1].set_title(f'Final Accuracy: {accuracies[-1] :.2f}%')
#ax[1].legend()
plt.show()

In [ ]:
len(train_data) 

In [ ]:
len(test_data)

In [ ]:
all_train_data = []
for _, (data, gt) in enumerate(iter(train_data)):
    all_train_data.append(data)
all_train_data = torch.stack(all_train_data)

In [ ]:
all_train_data.shape

In [ ]:
distances = []
for _, (data, gt) in enumerate(iter(test_loader)):
    for i in range(len(data)):
        min_dist = torch.min(torch.sum((all_train_data - data[i])**2, dim=(1, 2)))
        distances.append(min_dist)

In [ ]:
len(distances)

In [ ]:
min(distances)

In [ ]:
print(model)